In [1]:
import codecs
import sys
import numpy as np

In [2]:
path = '/Users/annanesterenko/Desktop/diplom_final'
sys.path.append(path)

In [18]:
k_1 = 2
b = 0.75

def count_word_freq(q, D):
    freq = 0
    for word in D:
        if word == q:
            freq += 1
            
    return freq

def idf(q, docs, count_docs):
    n = 0
    for doc in docs:
        if q in doc:
            n += 1
            
    return np.log((count_docs + 0.5) / (n + 0.5))

def bm25_score(Q, D, docs):   
    # avg len of docs in search demand
    sum_docs_len = 0
    count_docs = 0
    for doc in docs:
        sum_docs_len += len(doc)
        count_docs += 1
    avg_doc_len = sum_docs_len / count_docs
    
    score = 0
    for q_i in Q:
        freq = count_word_freq(q_i, D)
        score_q_i = freq * (k_1 + 1) / (freq + k_1 * (1 - b + b * len(D) / avg_doc_len))
        score_q_i *= idf(q_i, docs, count_docs)
        score += score_q_i

    return score

In [11]:
from CapsE.prepare_3 import *

In [12]:
lst_texts_query, lst_texts_doc, \
           train_duplets, test_duplets, \
           train_val_duplets, test_val_duplets = get_data_for_net_MIND_plain_text(
                                        texts_train=f'{path}/data/MIND/texts_train.txt', 
                                        texts_test=f'{path}/data/MIND/texts_test.txt',
                                        behaviors_train=f'{path}/data/MIND/behaviors_train.tsv',
                                        behaviors_test=f'{path}/data/MIND/behaviors_test.tsv')

In [17]:
len(test_duplets)

6000

In [49]:
def evaluate_quality(lst_texts_query, 
                     lst_texts_doc, 
                     test_duplets,
                     test_val_duplets):
    
    all_triplets = len(test_duplets)
    recall = 0
    recall1 = 0
    recall2 = 0
    recall3 = 0
    precision = 0
    
    ndcg3 = 0
    ndcg5 = 0
    ndcg10 = 0
    
    for i in range(len(test_duplets)):
        query = lst_texts_query[test_duplets[i][0][0]]
        answers = []
        
        for j in range(10):
            ans = lst_texts_doc[test_duplets[i][j][1]]
            answers.append(ans)
            
        scores = []
        relevant_cnt = 0
        for j in range(10):
            score = bm25_score(query, answers[j], answers)
            scores.append([score, test_val_duplets[i][j][0]])
            relevant_cnt += test_val_duplets[i][j][0]
            
        scores.sort(key=lambda x: x[0], reverse=True)
        scores_relevance = [i[1] for i in scores]
        
        precision += sum(scores_relevance[:relevant_cnt]) / relevant_cnt
        recall += (sum(scores_relevance[:relevant_cnt]) == relevant_cnt)
        recall1 += (scores_relevance[0] == 1)
        recall2 += (sum(scores_relevance[:2]) == 2)
        recall3 += (sum(scores_relevance[:3]) == 3)
        
        #ndcg@3
        ndcg3 += sum([scores_relevance[k-1] / np.log2(k+1) for k in range(1, 3)]) \
            / sum([1/np.log2(k+1) for k in range(1, 6)])
        #ndcg@5
        ndcg5 += sum([scores_relevance[k-1] / np.log2(k+1) for k in range(1, 6)]) \
            / sum([1/np.log2(k+1) for k in range(1, 6)])
        #ndcg@10
        ndcg10 += sum([scores_relevance[k-1] / np.log2(k+1) for k in range(1, 11)]) \
            / sum([1/np.log2(k+1) for k in range(1, 6)])
        
        
        if i % 100 == 0:
            print(i)
        
    return recall / all_triplets, precision / all_triplets, recall1 / all_triplets, \
            recall2 / all_triplets, recall3 / all_triplets, ndcg3 / all_triplets, \
            ndcg5 / all_triplets, ndcg10 / all_triplets,

In [50]:
evaluate_quality(lst_texts_query, lst_texts_doc, test_duplets, test_val_duplets)

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900


(0.0145,
 0.4158615079365095,
 0.42783333333333334,
 0.1785,
 0.06416666666666666,
 0.23669016246261756,
 0.4163153944519146,
 0.620464382702663)